In [2]:
from citipy import citipy
import time
from datetime import datetime

import requests
import json

from config import weather_api_key

from pprint import pprint


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

752

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Get the JSON text of the 'Get' request.
city_weather = requests.get(url).json()

pprint(city_weather)

{'cod': '400', 'message': 'Nothing to geocode'}


In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_des = [i['description'] for i in city_weather["weather"]][0]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          # "Date": city_date})
                          "Current Description": city_des})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | lumphat
Processing Record 2 of Set 1 | vaini
Processing Record 3 of Set 1 | berlevag
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | neuquen
Processing Record 6 of Set 1 | byron bay
Processing Record 7 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 8 of Set 1 | ushuaia
Processing Record 9 of Set 1 | sao joao da barra
Processing Record 10 of Set 1 | kavieng
Processing Record 11 of Set 1 | bluff
Processing Record 12 of Set 1 | voznesenye
Processing Record 13 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 14 of Set 1 | havelock
Processing Record 15 of Set 1 | cape town
Processing Record 16 of Set 1 | avarua
Processing Record 17 of Set 1 | bathsheba
Processing Record 18 of Set 1 | port alfred
Processing Record 19 of Set 1 | busselton
Processing Record 20 of Set 1 | vilhena
Processing Record 21 of Set 1 | guerrero negro
Processing Recor

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Lumphat,KH,13.4878,106.9816,72.05,99,100,2.17,overcast clouds
1,Vaini,TO,-21.2000,-175.2000,75.36,78,20,13.80,few clouds
2,Berlevag,NO,70.8578,29.0864,64.40,82,40,8.05,scattered clouds
3,Rikitea,PF,-23.1203,-134.9692,72.30,73,55,3.74,broken clouds
4,Neuquen,AR,-38.9516,-68.0591,62.49,8,0,0.00,clear sky
5,Byron Bay,AU,-28.6500,153.6167,59.70,62,19,7.83,few clouds
6,Ushuaia,AR,-54.8000,-68.3000,35.26,93,100,27.63,light rain
7,Sao Joao Da Barra,BR,-21.6403,-41.0511,66.42,77,99,29.91,overcast clouds
8,Kavieng,PG,-2.5744,150.7967,82.67,73,57,11.68,broken clouds
9,Bluff,NZ,-46.6000,168.3333,47.80,72,64,7.70,broken clouds


In [11]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")